# Pre-processing the Data

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import stat
import scipy.signal as scisig
import os
from scipy import stats
import scipy.stats

In [2]:
# %load cvxEDA.py
# https://github.com/lciti/cvxEDA

# Using cvxEDA for exploration of EDA signal
import numpy as np
import cvxopt as cv
import cvxopt.solvers

def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing
    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return (np.array(a).ravel() for a in (r, p, t, l, d, e, obj))

In [3]:
#Some of the sections from the below code is inspired from https://github.com/WJMatthew/WESAD


def wrist_data_extract(patient_data):
    patient_wrist_data = patient_data['signal']['wrist']
    patient_wrist_data.update({'Resp': patient_data['signal']['chest']['Resp']})
    return patient_wrist_data


# Pre-Processing Steps

def butter_lowpass_filter(data, cutoff, fs, order=6):
    nyq=fs/2
    new_cutoff=cutoff/nyq
    b, a = scisig.butter(order, new_cutoff, btype='low', analog=False)
    y = scisig.lfilter(b, a, data)
    return y

def convert_to_datetime(data, unit='s'):
    data.index = pd.to_datetime(data.index, unit=unit)
    return data

def eda_components(y):
    Fs = fs_dict['EDA']
    yn = (y - y.mean()) / y.std()
    [r, p, t, l, d, e, obj] = cvxEDA(yn, 1. / Fs)
    compo_list = [r, p, t, l, d, e, obj]
    return compo_list[:3]

def get_stats_features(data, label=-1):
    data_min = np.amin(data)
    data_max = np.amax(data)
    data_std = np.std(data)
    data_mean = np.mean(data)

    features = {'mean': data_mean, 'std': data_std, 'min': data_min, 'max': data_max,
                'label': label}
    return features

def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{savePath}{subject_feature_path}/S{s}_features7.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)

    df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
    df.drop(['0', '1', '2'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(f'{savePath}/combined_subjects7.csv')

    counts = df['label'].value_counts()
    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')

def get_peak_freq(x,fs=64):
    f, Pxx = scisig.welch(x,fs)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq

def find_peak_num(x):
    f,z=scipy.signal.find_peaks(x)
    n_peaks=len(f)
    return n_peaks
    
def get_slope(x):
    linreg = scipy.stats.linregress(np.arange(len(x)), x )
    slope = linreg[0]
    return slope


def get_window_samples(data, windows_number, label, length_of_window):
    global feat_names
    no_samples = []
    for i in range(windows_number):
        window_slice = data[length_of_window*i:length_of_window*(i+1)]
        window_statistics = get_stats_features(data=window_slice, label=label)
        temp_x = pd.DataFrame(window_statistics).drop('label', axis=0)
        temp_y = temp_x['label'][0]
        temp_x.drop('label', axis=1, inplace=True)

        if feat_names is None:
            feat_names = []
            for row in temp_x.index:
                for col in temp_x.columns:
                    feat_names.append('_'.join([row, col]))

        window_temp = pd.DataFrame(temp_x.values.flatten()).T
        window_temp.columns = feat_names
        window_temp = pd.concat([window_temp, pd.DataFrame({'label': temp_y}, index=[0])], axis=1)
        
        window_temp['BVP_peak_freq'] = get_peak_freq(window_slice['BVP'].dropna(),64)
        window_temp['TEMP_slope'] = get_slope(window_slice['TEMP'].dropna())
        window_temp['BVP_peaks_cnt']=find_peak_num(window_slice['BVP'].dropna())
        window_temp['Resp_peak_cnt']=find_peak_num(window_slice['Resp'].dropna())
        no_samples.append(window_temp)

    return pd.concat(no_samples)

In [4]:
WINDOW_IN_SECONDS = 10

In [5]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

In [6]:
for i in subject_ids:
    subject='S'+str(i)
    path="data/WESAD/"+subject+"/"+subject+".pkl"
    
    print("Processing for subject ",i)
    with open(path, 'rb') as file:
        patient_data = pickle.load(file, encoding='latin1')
        patient_label = patient_data['label']  

    signal_feat_dict = wrist_data_extract(patient_data)

    df_eda = pd.DataFrame(signal_feat_dict['EDA'], columns=['EDA'])
    df_bvp = pd.DataFrame(signal_feat_dict['BVP'], columns=['BVP'])
    df_temp = pd.DataFrame(signal_feat_dict['TEMP'], columns=['TEMP'])
    df_resp = pd.DataFrame(signal_feat_dict['Resp'], columns=['Resp'])
    
    
    
    df_label = pd.DataFrame(patient_label, columns=['label'])

    
    fs_dict = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'label': 700, 'Resp': 700}
    
    # Applying butterworth filter to EDA & Resp Signal
    df_eda['EDA'] = butter_lowpass_filter(df_eda['EDA'], 1.0, fs_dict['EDA'], 6)
    df_resp['Resp'] = butter_lowpass_filter(df_resp['Resp'], 1.0, fs_dict['Resp'], 6)
    

    # Adding signals with different sampling frequencies
    df_eda.index = [(1 / fs_dict['EDA']) * i for i in range(len(df_eda))]
    df_bvp.index = [(1 / fs_dict['BVP']) * i for i in range(len(df_bvp))]
    df_temp.index = [(1 / fs_dict['TEMP']) * i for i in range(len(df_temp))]
    df_label.index = [(1 / fs_dict['label']) * i for i in range(len(df_label))]
    df_resp.index = [(1 / fs_dict['Resp']) * i for i in range(len(df_resp))]

    # Converting the index values to time (datetime)

    df_eda = convert_to_datetime(df_eda, unit='s')
    df_bvp = convert_to_datetime(df_bvp, unit='s')
    df_temp = convert_to_datetime(df_temp, unit='s')
    df_label = convert_to_datetime(df_label, unit='s')
    df_resp = convert_to_datetime(df_resp, unit='s')

    # Applying cvxEDA on the EDA signal to derive features such as phasic, smna, tonic

    r_phasic, p_smna, t_tonic = eda_components(df_eda['EDA'])
    df_eda['EDA_phasic'] = r_phasic
    df_eda['EDA_smna'] = p_smna
    df_eda['EDA_tonic'] = t_tonic

    # Join the dataframes 
    df = df_eda.join(df_bvp, how='outer')
    df = df.join(df_temp, how='outer')
    df = df.join(df_resp, how='outer')
    df = df.join(df_label, how='outer')
    
    #Fill missing labels
    df['label'] = df['label'].fillna(method='ffill')

    df.reset_index(drop=True, inplace=True)

    # Normalizing the columns with min max normalization
    # df.columns
    df_label_1 = pd.DataFrame()
    df_label_1['label'] = df['label']
    df_temp = df.copy()

    #df = df.apply(standardize)
    df['label'] = df_temp['label'].copy()
    df_group = df.groupby('label')

    baseline = df_group.get_group(1) # get baseline data only
    stress = df_group.get_group(2) # get Stress data only
    amusement = df_group.get_group(3) # get amusement data only

    #get no of windows for window length=10 sec for each class
    baseline_wdws = int(len(baseline) / (fs_dict['label'] * WINDOW_IN_SECONDS)) 
    stress_wdws = int(len(stress) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    amusement_wdws = int(len(amusement) / (fs_dict['label'] * WINDOW_IN_SECONDS))

    label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0} # Other lables are not relevant for this study
    int_to_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
    feat_names = None
    savePath = 'data'
    subject_feature_path = '/SubjectData'

    samples_of_baseline = get_window_samples(baseline, baseline_wdws, 1, fs_dict['label'] * WINDOW_IN_SECONDS)
    samples_of_stress = get_window_samples(stress, stress_wdws, 2, fs_dict['label'] * WINDOW_IN_SECONDS)
    samples_of_amusement = get_window_samples(amusement, amusement_wdws, 0, fs_dict['label'] * WINDOW_IN_SECONDS)

    complete_samples = pd.concat([samples_of_baseline, samples_of_stress, samples_of_amusement])
    complete_samples = pd.concat([complete_samples.drop('label', axis=1), pd.get_dummies(complete_samples['label'])], axis=1)
    complete_samples.to_csv(f'{savePath}{subject_feature_path}/S{i}_features7.csv')

Processing for subject  2
     pcost       dcost       gap    pres   dres
 0: -1.2093e+04 -1.2039e+04  5e+04  2e+02  2e-01
 1: -1.2093e+04 -2.0218e+04  1e+04  4e+01  5e-02
 2: -1.2100e+04 -1.5246e+04  3e+03  1e+01  1e-02
 3: -1.2100e+04 -1.3551e+04  1e+03  4e+00  5e-03
 4: -1.2097e+04 -1.2876e+04  8e+02  2e+00  2e-03
 5: -1.2092e+04 -1.2599e+04  5e+02  8e-01  1e-03
 6: -1.2089e+04 -1.2320e+04  2e+02  3e-01  4e-04
 7: -1.2102e+04 -1.2158e+04  6e+01  2e-02  3e-05
 8: -1.2127e+04 -1.2148e+04  2e+01  6e-03  8e-06
 9: -1.2137e+04 -1.2146e+04  9e+00  2e-03  2e-06
10: -1.2142e+04 -1.2146e+04  4e+00  5e-04  6e-07
11: -1.2144e+04 -1.2145e+04  1e+00  1e-04  2e-07
12: -1.2145e+04 -1.2145e+04  5e-01  2e-05  3e-08
13: -1.2145e+04 -1.2145e+04  2e-01  5e-06  6e-09
14: -1.2145e+04 -1.2145e+04  7e-02  1e-06  2e-09
15: -1.2145e+04 -1.2145e+04  2e-02  3e-07  3e-10
16: -1.2145e+04 -1.2145e+04  5e-03  3e-08  4e-11
17: -1.2145e+04 -1.2145e+04  2e-03  6e-09  8e-12
18: -1.2145e+04 -1.2145e+04  4e-04  7e-10  9

18: -1.0437e+04 -1.0437e+04  3e-04  7e-11  2e-13
19: -1.0437e+04 -1.0437e+04  6e-05  7e-12  3e-14
20: -1.0437e+04 -1.0437e+04  2e-05  9e-13  2e-14
21: -1.0437e+04 -1.0437e+04  4e-06  5e-13  2e-14
Optimal solution found.
Processing for subject  10
     pcost       dcost       gap    pres   dres
 0: -1.0985e+04 -1.0958e+04  3e+04  2e+02  2e-01
 1: -1.0969e+04 -1.4118e+04  4e+03  2e+01  2e-02
 2: -1.0968e+04 -1.1518e+04  6e+02  3e+00  3e-03
 3: -1.0966e+04 -1.1164e+04  2e+02  8e-01  8e-04
 4: -1.0964e+04 -1.1036e+04  7e+01  2e-01  2e-04
 5: -1.0978e+04 -1.0991e+04  1e+01  6e-03  6e-06
 6: -1.0984e+04 -1.0990e+04  6e+00  2e-03  2e-06
 7: -1.0987e+04 -1.0990e+04  2e+00  5e-04  5e-07
 8: -1.0989e+04 -1.0990e+04  1e+00  2e-05  2e-08
 9: -1.0989e+04 -1.0990e+04  5e-01  7e-06  7e-09
10: -1.0990e+04 -1.0990e+04  2e-01  2e-06  2e-09
11: -1.0990e+04 -1.0990e+04  5e-02  4e-07  4e-10
12: -1.0990e+04 -1.0990e+04  1e-02  6e-08  5e-11
13: -1.0990e+04 -1.0990e+04  5e-03  9e-09  8e-12
14: -1.0990e+04 -1.

Combine the Files of different subjects into one main File

In [7]:
combine_files(subject_ids)

Number of samples per class:
baseline: 1907
stress: 1075
amusement: 598
